# Regex Test

Testing possible method of evaluating sentences for answers using Python Regexs

See the pos_tagger for what additional modules need to be installed

In [62]:
import re
import nltk
#from nltk import parse.stanford
from nltk import parse #need "requests" installed for using corenlp parse tagger

In [7]:
#Potential work with query reformulation
#See if this is more useful with that
query1 = "What is Gyrados most similar in appearance to?"
query2 = "Where is Gyrados first seen?" #This requires noting "appeared" and "seen" would be synonyms
query3 = "Where is a red Gyarados found in Pokémon Gold?"
query4 = "What franchise is Gyarados in?" #Note: the word "franchise" is lost in the regex, but would still be used in the query
query5 = "Who is Gyarados voiced by?"
query6 = "What is Gyarados known as?"
query7 = "How is the growth of Gyarados from Pokémon Diamond and Pearl to Pokémon Black and White described by IGN?"
#Note: Question could not start with wh-question word - eg. "Gyarados is voiced by who?"

In [37]:
p = re.compile('(Where |What |Who |How |Why |When )[\w|\s]*(is )([\w*|\s*]*)([?])')
queries = [query1, query2, query3, query4, query5, query6, query7]

listGroups = []
for q in queries:
    result = p.match(q)
    if result == None:
        #In actual usage, a flag would most likely be 
        #   set to indicate reformulation is not possible
        continue

    groups = result.groups()
    listGroups.append([groups[2]])
    #print(groups)
    #print(groups[2])
print(listGroups)
#Next step is to figure out where the "is" should be placed

[['Gyrados most similar in appearance to'], ['Gyrados first seen'], ['a red Gyarados found in Pokémon Gold'], ['Gyarados in'], ['Gyarados voiced by'], ['Gyarados known as'], ['the growth of Gyarados from Pokémon Diamond and Pearl to Pokémon Black and White described by IGN']]


In [38]:
def pos_tagger(docs):
    return nltk.pos_tag(docs)

#Look at the section of the question that does not contain wh-question, "is", or "?"
#Format for pos tagger into a list a words
listBreakdowns = []
for item in listGroups:
    for section in item:
        word_list = section.split(" ")

        #Remove excess "" if present in list
        #Temporary fix
        saved_index = -1
        for i in range(len(word_list)):
            if word_list[i] == "":
                saved_index = i
        
        if saved_index != -1:
            word_list.pop(saved_index)

        #print(word_list)
        breakdown = pos_tagger(word_list)
        listBreakdowns.append(breakdown)
        #print(breakdown)
print(listBreakdowns)

[[('Gyrados', 'NNP'), ('most', 'RBS'), ('similar', 'JJ'), ('in', 'IN'), ('appearance', 'NN'), ('to', 'TO')], [('Gyrados', 'NNP'), ('first', 'RB'), ('seen', 'VBN')], [('a', 'DT'), ('red', 'JJ'), ('Gyarados', 'NNP'), ('found', 'VBD'), ('in', 'IN'), ('Pokémon', 'NNP'), ('Gold', 'NNP')], [('Gyarados', 'NNP'), ('in', 'IN')], [('Gyarados', 'NNP'), ('voiced', 'VBN'), ('by', 'IN')], [('Gyarados', 'NNP'), ('known', 'VBN'), ('as', 'IN')], [('the', 'DT'), ('growth', 'NN'), ('of', 'IN'), ('Gyarados', 'NNP'), ('from', 'IN'), ('Pokémon', 'NNP'), ('Diamond', 'NNP'), ('and', 'CC'), ('Pearl', 'NNP'), ('to', 'TO'), ('Pokémon', 'NNP'), ('Black', 'NNP'), ('and', 'CC'), ('White', 'NNP'), ('described', 'VBN'), ('by', 'IN'), ('IGN', 'NNP')]]


In [68]:
#http://www.nltk.org/api/nltk.parse.html?highlight=stanford nltk.parse.stanford module
#Use parser.tagged_parse to get tree structure of sentence form
#   This is with the sentences tagged as above
#   parser.parse_sents can be used on a non-tokenized sentence TODO: See if there is any
#   performance difference between the two

#parser = StanfordParser(model_path="edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz")
#parser = parse.corenlp.CoreNLPParser(url='http://localhost:9000')
#parser.raw_parse('The quick brown fox jumps over the lazy dog.').pretty_print()
parser = parse.corenlp.GenericCoreNLPParser(url='http://localhost:9000', encoding='utf8')
next(parser.raw_parse('The quick brown fox jumps over the lazy dog.')).pretty_print()

AttributeError: 'GenericCoreNLPParser' object has no attribute 'parser_annotator'

In [39]:
#First search for adverb, then search for verb, then propositions, 
#   place "is" before first one found
#String remaining words together to form reformulated question
listReformulation = []
for phrase in listBreakdowns:
    reformulation = ""
    isPlaced = False
    for tag in ["RB", "VB", "IN"]: #Tags checked in this order
        reformulation = ""
        for taggedWord in phrase:
            if tag in taggedWord[1] and isPlaced == False:
                reformulation = reformulation + " is " + taggedWord[0]
                isPlaced = True
            else:
                if reformulation == "":
                    reformulation = taggedWord[0]
                else:
                    reformulation = reformulation + " " + taggedWord[0]
        if isPlaced == True:
            #Only have reformulation returned if all tags have 
            #   been searched or if a tag has been found
            break 

    listReformulation.append(reformulation)
print(listReformulation)

['Gyrados is most similar in appearance to', 'Gyrados is first seen', 'a red Gyarados is found in Pokémon Gold', 'Gyarados is in', 'Gyarados is voiced by', 'Gyarados is known as', 'the growth of Gyarados from Pokémon Diamond and Pearl to Pokémon Black and White is described by IGN']


In [ ]:
#TODO: Form query for each question

In [ ]:
#TODO: Open Pokemon text and compare query with each sentence
with open("noun_counting_data/a1.txt") as f:
    text = f.read()
    paragraphs = text.split("\n")
    sentences = []
    for entry in paragraphs:
        sentences.extend(entry.split("."))
print(sentences)